In [1]:
import requests
page = requests.get('https://www.basketball-reference.com/players/a/')
page

<Response [200]>

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
results = soup.find_all('th', class_="left")
str(results[0])

'<th class="left" data-append-csv="abdelal01" data-stat="player" scope="row"><a href="/players/a/abdelal01.html">Alaa Abdelnaby</a></th>'

In [4]:
import re

In [5]:
# pattern = 'data-append-csv="\w*\d\d"'

# match = re.findall(pattern, str(results[0]))
# type(match)

# match = re.findall(r'\w*\d\d', str(match))

# name = ''.join(match)

In [6]:
# gives us the ids for html scrapping
names = []
for result in results:
    result = str(result)
    match = re.findall(r'data-append-csv="\w*\d\d"', result)
    match = re.findall(r'\w*\d\d', str(match))
    name = ''.join(match)
    names.append(name)

In [7]:
# import pandas as pd

# dfs = []

# # compiling data for 'a' players
# for name in names:
#     try:
#         df = pd.read_html(f'https://www.basketball-reference.com/players/a/{name}.html')[0]
#         df['ID'] = name
#         dfs.append(df)
#     except:
#         pass

# # merging data together
# dfa = pd.concat(dfs)

In [8]:
def name_grabber(letter: str):
    '''
    Extracts basketball reference ID from html table and creates proper URL reference.
    '''
    page = requests.get(f'https://www.basketball-reference.com/players/{letter}/')
    
    soup = BeautifulSoup(page.content, 'html.parser')
    
    results = soup.find_all('th', class_="left")
    
    names = []
    
    for result in results:
        result = str(result)
        match = re.findall(r'data-append-csv="\w*\d\d"', result)
        match = re.findall(r'\w*\d\d', str(match))
        name = ''.join(match)
        names.append((name, f'https://www.basketball-reference.com/players/{letter}/{name}.html'))
        
    return names

In [9]:
letters = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()

results = [name_grabber(letter) for letter in letters]

In [10]:
# flattening list
results_flat = [y for x in results for y in x]
# https://coderwall.com/p/rcmaea/flatten-a-list-of-lists-in-one-line-in-python

In [11]:
import pandas as pd

dfs = []

for name, url in results_flat:
    try:
        df = pd.read_html(url)[0]
        df['ID'] = name
        df['URL'] = url
        dfs.append(df)
    except:
        pass

df = pd.concat(dfs)

In [16]:
# need to clean up the season col
df

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,PTS,ID,URL,3P,3PA,3P%,2P,2PA,2P%,eFG%
0,1968-69,22.0,TOT,NBA,C,49,NaN,19.3,2.9,8,...,7.3,abdulza01,https://www.basketball-reference.com/players/a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-69,22.0,CIN,NBA,NaN,20,NaN,5.4,0.9,2.2,...,1.9,abdulza01,https://www.basketball-reference.com/players/a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-69,22.0,MIL,NBA,C,29,NaN,28.9,4.3,12,...,11,abdulza01,https://www.basketball-reference.com/players/a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1969-70,23.0,MIL,NBA,PF,80,NaN,20.5,3,6.8,...,7.4,abdulza01,https://www.basketball-reference.com/players/a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970-71,24.0,SEA,NBA,C,61,NaN,20.9,4.3,9.8,...,10.9,abdulza01,https://www.basketball-reference.com/players/a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,zubaciv01,https://www.basketball-reference.com/players/z...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3 seasons,NaN,LAL,NBA,NaN,114,23,13.5,2.6,4.9,...,6.4,zubaciv01,https://www.basketball-reference.com/players/z...,0,0,0,2.6,4.8,0.545,0.541
9,2 seasons,NaN,LAC,NBA,NaN,98,95,18.9,3.4,5.8,...,8.6,zubaciv01,https://www.basketball-reference.com/players/z...,0,0,0,3.4,5.8,0.591,0.588
0,1948-49,29.0,WSC,BAA,NaN,56,NaN,NaN,1.8,5.8,...,4.9,zunicma01,https://www.basketball-reference.com/players/z...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# saving data as pickle so as to not run this again...
df.to_pickle('Data/nba_players.pkl')